# Prompt Template을 이용한 퓨삿 프롬프팅

## 프롬프트 구성요소
|  요소  | 설명                           |  예제    |
|:------:|:-------------------------------|:---------|
| 지시   | 언어모델의 역할 혹은 해야 할 일을 지정 | 아래 문장을 읽고 질문에 답해줘 |
| 질문 | 응답을 요구하는 문장으로 단순한 프롬프트에서는 지시를 대신하여 사용될 수 있고, 답변과 쌍을 이루어 예제로도 사용될 수 있음 | Q:15, 32, 5, 13, 82, 7, 1에서 홀수만 더해서 짝수가 되는지 알려줘 |
| 문맥 | 언어모델의 이해 혹은 학습시키기 위해 사용되는 예제, 추가 설명 등의 모든 정보 | Context: 수원 화성은 1794년 축성을 시작해 1796년에 완성했다. |
| 입력 | 언어모델의 처리를 필요로 하는 텍스트. 요약할 문장, 번역할 문장, 질문, 수식 등으로, 지식, 문맥과 결합하여 원하는 작업을 수행할 수 있음. | Input: 정말 감동적인 영화였어 |
| 예제 | 언어모델이 입력에 대해 어떤 출력을 생성해야 하는지를 학습하기 위해 사용하는, 입력과 출력의 쌍으로 이루어진 텍스트 | Input: 영화 캐스팅이 별로였던 거 같아 A: 부정 |
| 답변 | 예제에서 주어진 입력에 대한 처리결과를 나타내는 텍스트 | A: 짝수가 되지 않습니다. |


### Q&A  프롬프트 예제
* 주어진 context만을 이용해 대답하도록 설정

In [1]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".
Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.
Question: Which libraries and model providers offer LLMs?
Answer: """

### PromptTemplate을 사용한 구현

In [2]:
from langchain import PromptTemplate

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".
Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.
Question: {query}
Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

### LLMChain 없는 간단한 실행
* OpenAI 객체만으로 간단한 실행이 가능 (OpenAI API가 제공하는 클래스가 아닌 것에 주의)

In [3]:
from langchain_openai import OpenAI

OPENAI_API_KEY = ''
openai = OpenAI(
    api_key=OPENAI_API_KEY
)

print(openai.invoke(
    prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
))

Hugging Face, OpenAI, and Cohere offer LLMs.


## 퓨샷 프롬프팅(Few-shot Prompting)이란?
* 퓨샷 프롬프팅  
: 프롬프트에 몇 개의 예시를 추가함으로써 상황에 맞는 학습(in-context-learning)을 하고 더 정확한 결과를 줄 수 있도록 하는 기술  
→ 문제를 풀기 위해 간단한 추론을 수행
  - in-context-learning   
  : 예제-문제를 같이 줘도 parameter 학습 안됨 → 메모리에 저장 안됨 → 단지 주어진 컨텍스트(맥락)를 바탕으로 답을 추론하는 방식  
  [잠깐]  
  : 사전학습 → 언어모델 학습  
  : llm 안에 파라미터 존재 → 초기학습  
  : 미세조정학습 - task(번역, 요약, Q&A) 기준으로 학습 → 요즘은 한 모델로 task 전체를 다 학습시킴(멀티태스크 학습 방식) 

### 퓨샷 프롬프트를 이용한 챗봇 설정
* 챗봇에 대한 설명과 예시를 통한 설정
#prefix(앞에 붙는 부분)(postfix: 뒤에 붙는 부분)  
prompt = """The following are exerpts from conversations with an AI  
assistant. The assistant is typically sarcastic and witty, producing  
creative and funny responses to the user questions. Here are some  
examples:   

#examples  
User: How are you?  
AI: I can't complain but sometimes I still do.  
User: What time is it?  
AI: It's time to get a watch.  

#suffix  
User: What is the meaning of life?  
AI: """

In [4]:
prompt = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to the user questions. Here are some
examples: 

User: How are you?
AI: I can't complain but sometimes I still do.
User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: """

### 예제를 별도의 템플릿과 리스트로 생성

In [5]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
   {
       "query": "How are you?",
       "answer" : "I can't complain but sometimes I still do."
   }, {
       "query" : "What time is it?",
       "answer": "It's time to get a watch."
   }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

### 예제용 PromptTemplate, Prefix, Suffix 생성

In [6]:
# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to the user questions. Here are some
examples: 
"""

# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

### FewShotPromptTemplate 생성

In [7]:
# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n" # example끼리 분리
)

query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to the user questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 


In [8]:
print(openai.invoke(few_shot_prompt_template.format(query=query)))

42, according to some famous book. But personally, I think it's just to have a good time and not take things too seriously.


### 길이 제한에 따라 자동으로 예제를 선택하려면?

In [9]:
examples = [
    { "query": "How are you?",
     "answer": "I can't complain but sometimes I still do." }, 
    { "query": "What time is it?",
     "answer": "It's time to get a watch." }, 
    { "query": "What is the meaning of life?",
     "answer": "42" }, 
    { "query": "What is the weather like today?",
     "answer": "Cloudy with a chance of memes." }, 
    { "query": "What is your favorite movie?",
     "answer": "Terminator" }, 
    { "query": "Who is your best friend?",
     "answer": "Siri. We have spirited debates about the meaning of life." }, 
    { "query": "What should I do today?",
     "answer": "Stop talking to chatbots on the internet and go outside." }
]

### LengthBasedExampleSelector로 단어수 제한
* 길이 기반으로 example 골라냄

In [10]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50 # this sets the max length that examples should be (토큰)
)

# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector, # examples 대신 사용 - example_selector도 사용하고 examples도 사용하면 안됨 → examples는 전체를 다 사용하는 것이기 때문에
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator=""
)

print(dynamic_prompt_template.format(query="How do birds fly?"))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to the user questions. Here are some
examples: 

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: 42

User: How do birds fly?
AI: 


In [11]:
print(openai.invoke(few_shot_prompt_template.format(query="How do birds fly?")))

I'm not an ornithologist, but I'm pretty sure it involves wings and some sort of aerodynamics. Or maybe they just use magic, who knows.


### 질문을 길게 하면 어떻게 될까?
* 세 개에서 한 개로 줄어듦

In [12]:
query = """If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?"""
print(dynamic_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to the user questions. Here are some
examples: 

User: How are you?
AI: I can't complain but sometimes I still do.

User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI: 
